# Import PDK

## Import PDK in GDS format

To import a PDK from GDS format into gdsfactory you need 2 things:

- GDS file with all the cells that you want to import in the PDK (or separate GDS files, one per cell)
- Klayout layer properties files, to define the Layers that you can use when creating new custom Components.

GDS files are great for describing geometry thanks to the concept of References, where you store any geometry only once in memory.

For storing device metadata (settings, port locations, port widths, port angles ...) there is no clear standard.

`gdsfactory` stores the that metadata in `YAML` files, and also has functions to add pins

- `Component.write_gds()` saves GDS
- `Component.write_gds_metadata()` save GDS + YAML metadata

In [ ]:
import gdsfactory as gf

c = gf.components.mzi()
c

You can write **GDS** files only

In [ ]:
gdspath = c.write_gds("extra/mzi.gds")

Or **GDS** with **YAML** metadata information (ports, settings, cells ...)

In [ ]:
gdspath = c.write_gds_with_metadata("extra/mzi.gds")

This created a `mzi.yml` file that contains:
- ports
- cells (flat list of cells)
- info (function name, module, changed settings, full settings, default settings)

In [ ]:
c.metadata.keys()

You can read GDS files into gdsfactory thanks to the `import_gds` function

`import_gds` reads the same GDS file from disk without losing any information

In [ ]:
gf.clear_cache()

c = gf.import_gds(gdspath, read_metadata=True)
c

In [ ]:
import gdsfactory as gf

c2 = gf.import_gds(gdspath, name="mzi_sample", read_metadata=True)
c2

In [ ]:
c2.name

In [ ]:
c3 = gf.routing.add_fiber_single(c2)
c3

In [ ]:
gdspath = c3.write_gds_with_metadata("extra/pdk.gds")

In [ ]:
gf.labels.write_labels.write_labels_klayout(gdspath, layer_label=gf.LAYER.LABEL)

### add ports from pins

Sometimes the GDS does not have YAML metadata, therefore you need to figure out the port locations, widths and orientations.

gdsfactory provides you with functions that will add ports to the component by looking for pins shapes on a specific layers (port_markers or pins)

There are different pin standards supported to automatically add ports to components:

- PINs towards the inside of the port (port at the outer part of the PIN)
- PINs with half of the pin inside and half outside (port at the center of the PIN)
- PIN with only labels (no shapes). You have to manually specify the width of the port.


Lets add pins, save a GDS and then import it back.

In [ ]:
import gdsfactory as gf

c = gf.components.straight(
    decorator=gf.add_pins.add_pins
)  # add pins inside the component
c

In [ ]:
gdspath = c.write_gds("extra/wg.gds")

In [ ]:
gf.clear_cache()
c2 = gf.import_gds(gdspath)
c2

In [ ]:
c2.ports  # import_gds does not automatically add the pins

In [ ]:
c3 = gf.import_gds(gdspath, decorator=gf.add_ports.add_ports_from_markers_inside)
c3

In [ ]:
c3.ports

Foundries provide PDKs in different formats and commercial tools.

The easiest way to import a PDK into gdsfactory is to

1. have each GDS cell into a separate GDS file
2. have one GDS file with all the cells inside
3. Have a KLayout layermap. Makes easier to create the layermap.

With that you can easily create the PDK as as python package.

Thanks to having a gdsfactory PDK as a python package you can

- version control your PDK using GIT to keep track of changes and work on a team
    - write tests of your pdk components to avoid unwanted changes from one component to another.
    - ensure you maintain the quality of the PDK with continuous integration checks
    - pin the version of gdsfactory, so new updates of gdsfactory won't affect your code
- name your PDK version using [semantic versioning](https://semver.org/). For example patches increase the last number (0.0.1 -> 0.0.2)
- install your PDK easily `pip install pdk_fab_a` and easily interface with other tools



To create a **Python** package you can start from a customizable template (thanks to cookiecutter)

I usually create a python package by running this 2 commands inside a terminal

```
pip install cookiecutter
cookiecutter https://github.com/joamatab/cookiecutter-pypackage-minimal
```

It will ask you some questions to fill in the template (name of the package being the most important)


Then you can add the information about the GDS files and the Layers inside that package

In [ ]:
import gdsfactory as gf
from gdsfactory.layers import lyp_to_dataclass
from gdsfactory.config import PATH

In [ ]:
print(lyp_to_dataclass(PATH.klayout_lyp))

In [ ]:
# lets create a sample PDK (for demo purposes only) using GDSfactory
# if the PDK is in a commercial tool you can also do this. Make sure you save a single pdk.gds
sample_pdk_cells = gf.grid(
    [
        gf.components.straight,
        gf.components.bend_euler,
        gf.components.grating_coupler_elliptical,
    ]
)
sample_pdk_cells.write_gds("extra/pdk.gds")
sample_pdk_cells

In [ ]:
sample_pdk_cells.get_dependencies()

In [ ]:
# we write the sample PDK into a single GDS file
gf.clear_cache()
gf.write_cells.write_cells(gdspath="extra/pdk.gds", dirpath="extra/gds")

In [ ]:
# Lets generate the script that we need to have to each GDS cell into gdsfactory
import gdsfactory as gf

print(gf.write_cells.get_import_gds_script("extra/gds"))

You can also include the code to plot each fix cell in the docstring.

In [ ]:
print(gf.write_cells.get_import_gds_script("extra/gds", module="samplepdk.components"))

## Import PDK from other python packages

### phidl

To import a phidl pdk you can use the `gf.read.from_phidl`

### dphox

To import a dphox pdk you can use the `gf.read.from_dphox`

### picwriter

To import a picwriter pdk you can use the `gf.read.from_picwriter`

### nazca

To import a nazca pdk you can use the following code:

In [ ]:
from typing import Dict, Tuple
from gdsfactory.component import Component


def from_nazca(
    cell,
    layer_name_to_tuple: Dict[str, Tuple[int, int]],
    port_layer: Tuple[int, int] = (1, 0),
) -> Component:
    """Converts a nazca cell into a gdsfactory Component.

    Make sure you install nazca.

    Args:
        cell: nazca cell.
        layer_name_to_tuple: nazca layer name to gdslayer, gdspurpose.
        port_layer: layer for the ports.
    """
    import nazca as nd

    c = Component(cell.name)
    i = 0

    for named_tuple in nd.cell_iter(cell, flat=True):
        if named_tuple.cell_start:
            for polygon, points, bbox in named_tuple.iters["polygon"]:
                if polygon.layer == "bb_pin":
                    x0 = points[0][0]
                    y0 = points[0][1]

                    dx = points[1][0] - points[6][0]
                    dy = points[1][1] - points[6][1]

                    if dx > dy:
                        width = dx
                        if dx > 0:  # east
                            orientation = 90
                        else:
                            orientation = 270
                    else:
                        width = dy
                        if dy > 0:  # north
                            orientation = 180
                        else:
                            orientation = 0

                    name = f"o{i}"
                    c.add_port(
                        name=name,
                        center=(x0, y0),
                        width=width,
                        orientation=orientation,
                        layer=port_layer,
                    )
                    i += 1
                c.add_polygon(points, layer=layer_name_to_tuple[polygon.layer])

    c.auto_rename_ports()
    return c


def demo():
    import nazca as nd
    import nazca.demofab as demo
    import pandas as pd

    n = demo.deep.euler(angle=90).put()
    df = pd.read_csv(demo.layer_file)

    layer_name_to_tuple = dict(
        Deep=(1, 0),
        DeepTrench=(4, 0),
        bb_pin=(501, 0),
    )
    c = from_nazca(n, layer_name_to_tuple=layer_name_to_tuple)
    c.show(show_ports=True)

## Foundry PDKs

You can build PDKs for different foundries. PDKs contain some foundry IP (layer stack, Design Rules) and are available only under and NDA.
For accessing the gdsfactory PDK please contact your foundry for access.


Some PDKs are open source and publicly available:

- [ubcpdk](https://github.com/gdsfactory/ubc)
- [sky130](https://github.com/gdsfactory/skywater130)


## Build your own PDK

You can create a PDK as a python library using a cookiecutter template. For example, you can use this one.

```
pip install cookiecutter
cookiecutter https://github.com/joamatab/cookiecutter-pypackage-minimal
```

Or you can fork the ubcpdk and create new PCell functions that use the correct layers for your foundry. For example.

```

from pydantic import BaseModel


class LayerMap(BaseModel):
    WGCORE = (3, 0)
    LABEL = (100, 0)
    DEVREC: Layer = (68, 0)
    LABEL: Layer = (10, 0)
    PORT: Layer = (1, 10)  # PinRec
    PORTE: Layer = (1, 11)  # PinRecM
    FLOORPLAN: Layer = (99, 0)

    TE: Layer = (203, 0)
    TM: Layer = (204, 0)
    TEXT: Layer = (66, 0)
    LABEL_INSTANCE: Layer = (66, 0)


LAYER = LayerMap()

```
